We will dissect each part of the backtester to see how they work.
# 1. The DataHandler

In [1]:
from data_handler import HistoricalPolygonDataHandler
from polygon.tickers import get_id
from datetime import datetime, date
from event import MarketEvent
import queue
import pandas as pd

In [2]:
events = queue.Queue()

In [3]:
data_handler = HistoricalPolygonDataHandler(events)
data_handler.load_data("AAPL", start=date(2023, 8, 1), end=date(2023, 9, 1))
data_handler.load_data("AA", start=date(2023, 7, 1), end=date(2023, 9, 1))

In [4]:
data_handler.get_loaded_symbols()

['AAPL', 'AA']

In [5]:
for i in range(10):  
    data_handler.update_bars(datetime(2023, 8, 1, hour=9, minute=30+i))

In [6]:
len(data_handler._latest_bars["AAPL"])

10

In [7]:
data_handler.get_latest_bars("AAPL", N=2)

,open,high,low,close,close_original,volume,tradeable,halted
2023-08-01 09:38:00,196.3950,196.49,196.3871,196.43,196.43,128663,True,False
2023-08-01 09:39:00,196.4205,196.59,196.3700,196.55,196.55,235643,True,False


# 2. Broker

In [8]:
from broker import SimulatedBroker
from event import OrderEvent
broker = SimulatedBroker(events, data_handler)
order = OrderEvent(datetime(2023, 8, 1, hour=9, minute=39), "AAPL", side="BUY", quantity=10)

2023-08-01T09:39:00 | ORDER BUY 10 of AAPL


In [9]:
broker.execute_order(order)

In [10]:
for i in range(10):
    events.get() # Remove the MarketEvents
event = events.get()
event

In [11]:
event.total_fill

1965.5

# 3. Portfolio

In [12]:
from portfolio import StandardPortfolio
from event import FillEvent
from data_handler import HistoricalPolygonDataHandler
from polygon.tickers import get_id
from datetime import datetime, date
import queue
events = queue.Queue()

In [13]:
data_handler = HistoricalPolygonDataHandler(events)
data_handler.load_data("AAPL", start=date(2023, 8, 1), end=date(2023, 9, 1))
data_handler.load_data("AA", start=date(2023, 7, 1), end=date(2023, 9, 1))

portfolio = StandardPortfolio(events, data_handler, start_date=datetime(2023, 8, 1, hour=9, minute=30))

In [14]:
portfolio.current_equity

10000.0

In [15]:
portfolio.current_positions_value

0

In [16]:
for i in range(1, 5):  
    data_handler.update_bars(datetime(2023, 8, 1, hour=9, minute=30+i))
    portfolio.append_portfolio_log(dt=datetime(2023, 8, 1, hour=9, minute=30+i))
data_handler.get_latest_bars("AAPL", N=2)

,open,high,low,close,close_original,volume,tradeable,halted
2023-08-01 09:33:00,196.05,196.1900,195.950,195.9658,195.9658,124321,True,False
2023-08-01 09:34:00,195.95,196.3961,195.925,196.3900,196.3900,161667,True,False


In [17]:
fill = FillEvent(dt=datetime(2023, 8, 1, hour=9, minute=34), symbol="AAPL", side='BUY', quantity=25, fill_price=196.39, commission=100)
portfolio.update_from_fill(fill)

In [18]:
fill = FillEvent(dt=datetime(2023, 8, 1, hour=9, minute=34), symbol="AAPL", side='BUY', quantity=25, fill_price=196.39, commission=100)
portfolio.update_from_fill(fill)

In [19]:
portfolio.current_positions

{'AAPL': 50}

In [20]:
portfolio.current_cash

-19.5

In [21]:
portfolio.current_positions_value

9819.5

In [22]:
portfolio.current_equity

9800.0

In [23]:
for i in range(5):  
    data_handler.update_bars(datetime(2023, 8, 1, hour=9, minute=35+i))
    portfolio.append_portfolio_log(dt=datetime(2023, 8, 1, hour=9, minute=35+i))
data_handler.get_latest_bars("AAPL", N=2)

,open,high,low,close,close_original,volume,tradeable,halted
2023-08-01 09:38:00,196.3950,196.49,196.3871,196.43,196.43,128663,True,False
2023-08-01 09:39:00,196.4205,196.59,196.3700,196.55,196.55,235643,True,False


In [24]:
fill = FillEvent(dt=datetime(2023, 8, 1, hour=9, minute=39), symbol="AAPL", side='SELL', quantity=60, fill_price=196.55, commission=100)
portfolio.update_from_fill(fill)

In [25]:
for i in range(5):  
    data_handler.update_bars(datetime(2023, 8, 1, hour=9, minute=40+i))
    portfolio.append_portfolio_log(dt=datetime(2023, 8, 1, hour=9, minute=40+i))
data_handler.get_latest_bars("AAPL", N=2)

,open,high,low,close,close_original,volume,tradeable,halted
2023-08-01 09:43:00,196.4814,196.62,196.45,196.5350,196.5350,93577,True,False
2023-08-01 09:44:00,196.5300,196.56,196.42,196.4798,196.4798,83679,True,False


In [26]:
data_handler.update_bars(datetime(2023, 8, 1, hour=9, minute=45))
fill = FillEvent(dt=datetime(2023, 8, 1, hour=9, minute=45), symbol="AAPL", side='BUY', quantity=10, fill_price=196.50, commission=100)
portfolio.update_from_fill(fill)
portfolio.append_portfolio_log(dt=datetime(2023, 8, 1, hour=9, minute=45))

(I prefer only logging the performance each day instead of every bar, else this list gets unnecessarily long)

In [27]:
portfolio.create_df_from_holdings_log()

,equity,cash,positions_value,positions,returns,returns_cum
datetime,,,,,,
2023-08-01 09:30:00,10000.000,10000.0,0.000,{},0.000000,0.000000
2023-08-01 09:31:00,10000.000,10000.0,0.000,{},0.000000,0.000000
2023-08-01 09:32:00,10000.000,10000.0,0.000,{},0.000000,0.000000
2023-08-01 09:33:00,10000.000,10000.0,0.000,{},0.000000,0.000000
2023-08-01 09:34:00,10000.000,10000.0,0.000,{},0.000000,0.000000
2023-08-01 09:35:00,9792.500,-19.5,9812.000,{'AAPL': 50},-0.020750,-0.020750
2023-08-01 09:36:00,9799.500,-19.5,9819.000,{'AAPL': 50},0.000715,-0.020050
2023-08-01 09:37:00,9800.500,-19.5,9820.000,{'AAPL': 50},0.000102,-0.019950
2023-08-01 09:38:00,9802.000,-19.5,9821.500,{'AAPL': 50},0.000153,-0.019800


In [33]:
trade_log = pd.DataFrame(portfolio.transaction_log)
trade_log.set_index('datetime', inplace=True)
trade_log

,symbol,side,quantity,fill_price,commission
datetime,,,,,
2023-08-01 09:34:00,AAPL,BUY,25,196.39,100
2023-08-01 09:34:00,AAPL,BUY,25,196.39,100
2023-08-01 09:39:00,AAPL,SELL,60,196.55,100
2023-08-01 09:45:00,AAPL,BUY,10,196.50,100


# 4. Trade log
The trade log is the list of profit/losses from every trade. All information we need to calculate that are in the transaction log. However this is not straightforward. If you look at the transaction log above, what are the trades? Should the BUY 25 from the first 2 trades be grouped? If the short trade was 40 shares instead, how would be assign them to the first 2 trades? Do we see that as closing the the first and partially closing the second? What about positions that are still open?

When constructing the trade log, we use the following rules:
1. We never group opening trades. So we see the first 2 trades as seperate trades.
2. Trades in the opposing direction are assigned as FIFO. If the opposing direction is larger than the entire posiiton, we see this as a new position. So the SELL -60 means we exit the 1st and 2nd trade, and create a new trade that is short 10 shares. If the trade was SELL -40 instead, we would assign -25 to the first trade and -15 to the second trade. Because we use FIFO.
3. For open positions, we can calculate the current P/L.

In [35]:
def calculate_trade_log(transaction_log):
    pass